In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
import yfinance as yf
from functools import reduce
import operator
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
data = yf.download("AAPL", period = "60d", interval="5m")  # For minute-level bars
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
df = data.dropna()
df = df[~pd.isnull(df['Close'])].copy()
df.reset_index(inplace=True)
df.columns = df.columns.get_level_values(0)

df['range'] = df['High'] - df['Low']
df['body_length'] = abs(df['Close'] - df['Open'])
df['upper_tail'] = df['High'] - df[['Close','Open']].apply(lambda x: max(x), axis=1)
df['lower_tail'] = df[['Close','Open']].apply(lambda x: min(x), axis=1) - df['Low']
df['body_length/range'] = df['body_length'] / df['range']
df['upper_tail/range'] = df['upper_tail'] / df['range']
df['lower_tail/range'] = df['lower_tail'] / df['range']
df['return'] = df['Close'].pct_change()
df['local_vol'] = df['return'].rolling(window=10, min_periods=3).std()
df['rolling_mean_volume'] = df['Volume'].rolling(window = 10, min_periods = 3).mean()
df['volume_spike'] = df[['Volume','rolling_mean_volume']].apply(lambda x: x[0]/x[1] if ( not any(np.isnan([x[0] ,x[1] ]) )) else None, axis=1 )
print(df.shape)

df = df.dropna()
print(df.shape)
df.reset_index(inplace=True)
df = df.iloc[:,1:]
print(df.shape)
df[:3]

[*********************100%***********************]  1 of 1 completed
<ipython-input-25-924508244>:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['volume_spike'] = df[['Volume','rolling_mean_volume']].apply(lambda x: x[0]/x[1] if ( not any(np.isnan([x[0] ,x[1] ]) )) else None, axis=1 )


(4678, 17)
(4675, 17)
(4675, 17)


Price,Datetime,Open,High,Low,Close,Volume,range,body_length,upper_tail,lower_tail,body_length/range,upper_tail/range,lower_tail/range,return,local_vol,rolling_mean_volume,volume_spike
0,2025-03-20 13:45:00+00:00,214.604996,214.958801,214.479996,214.660004,475659,0.478806,0.055008,0.298798,0.12500,0.114886,0.624048,0.261066,0.000140,0.002988,869081.5,0.547312
1,2025-03-20 13:50:00+00:00,214.649994,215.160004,214.520004,215.154999,418795,0.639999,0.505005,0.005005,0.12999,0.789071,0.007820,0.203109,0.002306,0.002548,779024.2,0.537589
2,2025-03-20 13:55:00+00:00,215.149994,215.490005,214.847504,215.490005,480799,0.642502,0.340012,0.000000,0.30249,0.529199,0.000000,0.470801,0.001557,0.002212,729320.0,0.659243


Consider 30 rolling pts for training data, and summarise the features across 30 - 5m candles to get a single points for training.

In [26]:
window = 10  # Considering 10 such 5m candles and collapsing into a single data that will represent this window
look_ahead_period = 3
# i =0
# d = df[i:i+window]


#avg_columns = ['average_range','average_body_length','average_upper_tail','average_lower_tail','average_body_length/range','average_upper_tail/range','average_lower_tail/range','average_return']
extra_cols = ['Green_candle_perc' , 'breakout_up', 'breakout_down' , '2ndHalf_1stHalf_return_diff', 'trend_slope', 'cumulative_return', 'compounded_return', 'rolling_'+str(window) + '_vol' , 'coeff_of_variation' , 'volume_trend_slope', 'secondHalf_volume_average', 'price_to_volume_correlation' ]
x_train = pd.DataFrame(columns = extra_cols)
y_train = []
y_train_threshold = []
d = df.copy()
for i in tqdm(range(0, len(df) - window )):

  d = df[i:i+window].copy()
  d.reset_index(inplace=True)

  avg_future_closing_price = (df['Close'].values[i+window :  i+window + look_ahead_period]).mean()

  data = pd.DataFrame(columns = x_train.columns)

  # Average statistics
  #data.loc[i,avg_columns] = d.loc[:,['range','body_length','upper_tail','lower_tail','body_length/range','upper_tail/range','lower_tail/range','return']].mean().values

  # Price_action features
  data.loc[i,'Green_candle_perc'] = sum(d.loc[:window,['Open', 'Close']].apply(lambda x: x[1] > x[0], axis=1))/window
  data.loc[i, 'breakout_up'] = 1 if d.loc[window-1,'Close'] > max(d['Close'].values[:-1]) else 0
  data.loc[i, 'breakout_down'] = 1 if d.loc[window-1,'Close'] < min(d['Close'].values[:-1]) else 0

  # Momentum features
  data.loc[i, '2ndHalf_1stHalf_return_diff'] = (d.loc[(window/2):window,'return'].values - d.loc[:(window/2)-1,'return'].values ).mean()
  data.loc[i, 'trend_slope'] =   np.polyfit(np.arange(0,window), d['Close'].values, 1)[0]
  data.loc[i, 'cumulative_return'] =  (d.loc[:,'Close'][1]/ d.loc[:,'Close'][0] ) - 1
  data.loc[i, 'compounded_return'] = pow(reduce(operator.mul, list(1 + d['return'] ) ),1/window ) - 1

  # Volatility features
  data.loc[i, 'rolling_'+str(window) + '_vol'] = d.loc[:, 'return'].std()
  data.loc[i, 'coeff_of_variation'] =  d.loc[:, 'return'].std() / d.loc[:, 'return'].mean()

  # Volume features
  data.loc[i, 'volume_trend_slope'] = np.polyfit(np.arange(window/2), d.loc[(window)/2 :i+window, 'Volume'].values, 1)[0] /  d.loc[ (window)/2 :window, 'Volume'].mean()
  data.loc[i, 'secondHalf_volume_average'] = (d.loc[(window)/2 :window, 'Volume'].values).mean() / d.loc[:, 'Volume'].mean()

  # Price_to_Volume Correlation
  data.loc[i, 'price_to_volume_correlation'] = np.corrcoef( d.loc[:,'Close'], d.loc[:,'Volume'] )[0,1]

  x_train = pd.concat([x_train, data],axis=0, ignore_index=True)

  y_train.append( 1 if avg_future_closing_price > d.loc[window-1,'Close'] else 0)

  y_train_threshold.append( 1 if avg_future_closing_price > d.loc[window-1,'Close'] * 1.005 else -1 if avg_future_closing_price < d.loc[window-1,'Close'] * 0.995 else 0)
data

Streaming output truncated to the last 5000 lines.
<ipython-input-26-1796921482>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.loc[i,'Green_candle_perc'] = sum(d.loc[:window,['Open', 'Close']].apply(lambda x: x[1] > x[0], axis=1))/window
<ipython-input-26-1796921482>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.loc[i,'Green_candle_perc'] = sum(d.loc[:window,['Open', 'Close']].apply(lambda x: x[1] > x[0], axis=1))/window
<ipython-input-26-1796921482>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (con

,Green_candle_perc,breakout_up,breakout_down,2ndHalf_1stHalf_return_diff,trend_slope,cumulative_return,compounded_return,rolling_10_vol,coeff_of_variation,volume_trend_slope,secondHalf_volume_average,price_to_volume_correlation
4664,0.4,0,1,-0.000133,-0.059035,-0.000458,-0.000168,0.000792,-4.717608,0.214631,1.268985,-0.7485


In [27]:
print(pd.Series(y_train).value_counts())
print(pd.Series(y_train_threshold).value_counts())
x_train.shape, len(y_train), len(y_train_threshold)

1    2334
0    2331
Name: count, dtype: int64
 0    4296
-1     201
 1     168
Name: count, dtype: int64


((4665, 12), 4665, 4665)

Check for multi-collinearity

In [28]:
from sklearn.model_selection import train_test_split
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Example: df is your DataFrame of features
x = add_constant(x_train)  # adds intercept term
x = x.astype(float)

vif_data = pd.DataFrame()
vif_data['feature'] = x.columns
vif_data['VIF'] = [variance_inflation_factor(x.values, i) for i in range(x.shape[1])]

print(vif_data)

                        feature        VIF
0                         const  37.149257
1             Green_candle_perc   1.502366
2                   breakout_up   1.291238
3                 breakout_down   1.339484
4   2ndHalf_1stHalf_return_diff   1.345212
5                   trend_slope   6.236378
6             cumulative_return   1.306485
7             compounded_return   6.953228
8                rolling_10_vol   1.019506
9            coeff_of_variation   1.000499
10           volume_trend_slope   1.151696
11    secondHalf_volume_average   1.098366
12  price_to_volume_correlation   1.084551


In [38]:
x_train.dtypes

,0
Green_candle_perc,object
breakout_up,object
breakout_down,object
2ndHalf_1stHalf_return_diff,object
trend_slope,object
cumulative_return,object
compounded_return,object
rolling_10_vol,object
coeff_of_variation,object
volume_trend_slope,object


In [45]:
from sklearn.model_selection import train_test_split

X_tr, X_test ,Y_tr, Y_test = train_test_split(x_train, y_train_threshold, test_size = 0.3, shuffle=True, random_state = 42)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from lightgbm import LGBMClassifier

x_train = x_train.astype(float)

strat_kFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = LGBMClassifier(objective='multiclass', num_class=3, random_state=42)

x_tr, x_cv ,y_tr, y_cv = [], [], [], []

for train_idx, cv_idx in strat_kFold.split(X_tr, Y_tr):

  x_tr, x_cv = x_train.loc[train_idx,:], x_train.loc[cv_idx,:]
  y_tr, y_cv = [y_train_threshold[i] for i in train_idx], [y_train_threshold[i] for i in cv_idx]


  model.fit(x_tr, y_tr)

  y_pred = model.predict(x_cv)

  print("Accuracy:", accuracy_score(y_cv, y_pred))
  print(classification_report(y_cv, y_pred))